In [2]:
# Instalar libreria, solo si no se ha instalado previamente
# pip install alpha_vantage

In [9]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

In [13]:
# Importar API key guardada en un archivo separado
import sys
sys.path.append('../')  # Añade el directorio principal al sys.path
from OTROS.credentials import api_key

In [14]:
# Símbolo de la acción que deseas obtener
ticker = "VOO"

In [15]:
# Crea una instancia de la clase TimeSeries con tu clave de API
ts = TimeSeries(key=api_key)

# Obtiene datos históricos de precios de acciones
datos, meta_datos = ts.get_daily(symbol=ticker, outputsize='full')

In [16]:
df = pd.DataFrame.from_dict(datos, orient='index')
df.columns = ['open', 'high', 'low', 'close', 'volume']
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

In [17]:
df.head()

,open,high,low,close,volume
2023-12-21,433.13,434.640,430.8552,434.28,4621606.0
2023-12-20,435.51,437.305,429.9000,430.09,5783234.0
2023-12-19,436.04,438.220,435.9500,438.19,4037929.0
2023-12-18,434.64,436.400,434.4500,435.54,4280475.0
2023-12-15,433.19,434.250,432.2400,433.09,5326041.0
